### Import libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spo

### Define function for loading data

In [3]:
def load_set(name):
    dates = pd.date_range('2015-10-03', '2016-10-02')
    df = pd.DataFrame(index=dates)
    shares = pd.read_csv(name+'.csv', index_col='Date', parse_dates=True, usecols=['Date', 'Adj Close'])
    df = df.join(shares)
    df = df.dropna(subset=["Adj Close"])
    return df.copy()

### Define function for calculating sharp ratio

In [4]:
def get_sharp_ratio(df, w, x, y, z):

    start_val = 1000000
    #allocate
    df['spy'] = df['spy']*w
    df['xom'] = df['xom']*x
    df['goog'] = df['goog']*y
    df['gld'] = df['gld']*z

    #position values
    df[['spy', 'xom', 'goog', 'gld']] = df[['spy', 'xom', 'goog', 'gld']]*start_val


    #daily portfolio values
    df['port_val'] = df.sum(axis=1)

    #daily returns
    df['daily_rets'] = df['port_val'] - start_val
    df['daily_rets'] = df['daily_rets'][1:]

    cum_ret = (df['port_val'][-1]/df['port_val'][0])-1
    avg_daily_ret = df['daily_rets'].mean()
    std_daily_ret = df['daily_rets'].std()

    #sharp ratio
    sharp_ratio = np.sqrt(252)*(avg_daily_ret/std_daily_ret)
    return sharp_ratio

### Define function for optimization

In [5]:
def f(c):
    spy = load_set('spy')
    xom = load_set('xom')
    goog = load_set('goog')
    gld = load_set('gld')

    #merge
    df = pd.DataFrame(index=spy.index)
    df['spy'] = spy['Adj Close']
    df['xom'] = xom['Adj Close']
    df['goog'] = goog['Adj Close']
    df['gld'] = gld['Adj Close']

    #normalize
    df['spy'] = df['spy']/df['spy'][0]
    df['xom'] = df['xom']/df['xom'][0]
    df['goog'] = df['goog']/df['goog'][0]
    df['gld'] = df['gld']/df['gld'][0]

    return get_sharp_ratio(df.copy(), c[0], c[1], c[2], c[3])*-1

### Calling optimizer

In [6]:
Xguess = [0.25,0.25,0.25,0.25]
#sum of all variables must be 1
constraints = ({'type': 'eq', 'fun': lambda x: sum(x)-1})
bounds = [(0, 1)]*4
min_result = spo.minimize(f, Xguess, method="SLSQP", constraints=constraints, bounds=bounds, options={'disp': True})
print 'Minima found at:'
print "spy = {}, xom = {}, goog = {}, gld = {}, sharp_ratio = {}".format(min_result.x[0], min_result.x[1], min_result.x[2], min_result.x[3], min_result.fun*-1)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -45.945568968
            Iterations: 6
            Function evaluations: 36
            Gradient evaluations: 6
Minima found at:
spy = 0.0, xom = 0.255442582088, goog = 0.744557417913, gld = 0.0, sharp_ratio = 45.945568968
